In [1]:
import random
%load_ext autoreload
%autoreload 2

In [2]:
import ast
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from pprint import pprint
from collections import Counter
from galleries_mapping import *
import matplotlib.pyplot as plt
from sklearn.utils import resample

In [3]:
def plot_tags_count(data: pd.DataFrame | dict):
    if isinstance(data, pd.DataFrame):
        data = Counter([item for sublist in data['labels'] for item in sublist])
    sorted_values_cntr = {k: v for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True)}
    plt.figure(figsize=(10, 10))
    plt.pie(
        sorted_values_cntr.values(),
        labels=sorted_values_cntr.keys(),
        autopct='%1.1f%%', startangle=0
    )
    plt.axis('equal')
    plt.title('Label Distribution in Test Set')

    plt.show()


def plot_variance_per_key(data: dict):
    plt.figure(figsize=(10, 10))
    plt.bar(data.keys(), data.values())
    plt.title('Label Distribution in Test Set')
    plt.show()

In [4]:
SRC_DIR = Path('/Volumes/external_drive')
FILTERED_PORNHUB_CATEGORIES = [
    'anal',
    'bbw',
    'big ass',
    'big dick',
    'big tits',
    'blonde',
    'blowjob',
    'bondage',
    'brunette',
    'cosplay',
    'creampie',
    'cumshot',
    'double penetration',
    'ebony',
    'feet',
    'fingering',
    'fisting',
    'handjob',
    'hardcore',
    'lesbian',
    'massage',
    'masturbation',
    'milf',
    'old/young',
    'pissing',
    'public',
    'pussy licking',
    'red head',
    'rough sex',
    'small tits',
    'smoking',
    'solo',
    'squirt',
    'strap on',
    'striptease',
    'tattooed women',
    'teen',
    'threesome',
    'toys',
    'transgender'
]

try:
    _df = pd.read_csv('datasets/images_high_res_dataset.csv')
except FileNotFoundError:
    _df = pd.read_csv(SRC_DIR / 'images_high_res_dataset.csv')

print("Dataset loaded.")
_df.drop(['models'], axis=1, inplace=True)
_df['categories'] = _df['categories'].apply(ast.literal_eval)
_df['categories_suggestions'] = _df['categories_suggestions'].apply(ast.literal_eval)
print("Parsed list columns categories")

Dataset loaded.
Parsed list columns categories


In [5]:
df = _df.copy(True)
df.shape

(1345430, 7)

# Merge suggestions and categories

In [6]:
def merge_categories(row):
    categories = set(row['categories'])
    categories_suggestions = set(row['categories_suggestions'])
    categories_superset = {category.strip().lower() for category in categories.union(categories_suggestions)}
    return list(categories_superset)


df['labels'] = df['categories'].apply(lambda x: [category.strip().lower() for category in x])
df['labels'] = df.apply(merge_categories, axis=1)
df.drop(['categories_suggestions', 'categories'], axis=1, inplace=True)
df.shape

(1345430, 6)

# Purge nationalities from tags

In [7]:
nationality_tags_to_purge = {
    'african',
    'american',
    'arab',
    'argentina',
    'australian',
    'brazilian',
    'british',
    'canadian',
    'chinese',
    'colombian',
    'cuban',
    'czech',
    'dutch',
    'european',
    'filipina',
    'french',
    'german',
    'hungarian',
    'indian',
    'italian',
    'japanese',
    'korean',
    'mexican',
    'pinay',
    'polish',
    'russian',
    'spanish',
    'thai',
    'ukrainian',
    'venezuela',
    'white'
}
nationality_tags_to_purge = {n.lower() for n in nationality_tags_to_purge}
df['labels'] = df.labels.apply(lambda x: list(set(x) - nationality_tags_to_purge))
df.shape

(1345430, 6)

# Apply gallery mapping

In [8]:
def gallery_mapping(row):
    labels = row.labels

    out = []
    for L in labels:
        _fetched = GALLERIES_MAP.get(L, None)
        if _fetched is remove_tag:
            continue
        elif _fetched is remove_gallery:
            return None
        elif isinstance(_fetched, list):
            out.extend(_fetched)
        elif _fetched is keep_tag:
            out.append(L.lower())

    return list(set(out))


df['labels'] = df.apply(gallery_mapping, axis=1)
df = df[df['labels'].notnull()]
df.shape

(1114113, 6)

In [ ]:
fp = 'thai/chubby-amateur-babe-nikki-shows-her-juicy-bosom-hot-pussy-before-giving-a-bj-95099173'
filtered_df = df[df['file_path'].str.startswith(fp)]
filtered_df

# Dataframe clean up

In [ ]:
df['file_path'] = df['gallery_category'] + '/' + df['gallery_name'] + '/' + df['filename']
df = df.drop(
    ['gallery_category', 'gallery_name', 'filename'], axis=1
)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.drop(
    ['height', 'width'], axis=1, inplace=True
)

In [ ]:
df.shape

In [ ]:
# df.to_csv('datasets/all_file_paths.csv')

# Balancing the dataset

In [ ]:
label_counts = Counter(item for sublist in df['labels'] for item in sublist)
ascending_labels = [
    k for k, v in sorted(label_counts.items(), key=lambda item: item[1])
]
label_proportions = {
    k: v / len(df) for k, v in sorted(label_counts.items(), key=lambda item: item[1])
}

In [ ]:
ascending_labels

In [ ]:
from prompts.prompts import dynamic_prompt

print(dynamic_prompt())

In [ ]:
# ['big dick', 'bbc', 'double penetration', 'ssbbw', 'footjob', 'toys', 'butt plug', 'squirting', 'sandals', 'missionary', 'fisting', 'teacher', 'threesome', 'rimming', 'ass licking', 'group sex', 'pissing', 'deepthroat', 'orgasm', 'gym', 'beach', 'shower', 'massage', 'interracial', 'cameltoe', 'pants', 'granny', 'titjob', 'cosplay', 'thong', 'smoking', 'maid', 'cum in mouth', 'creampie', 'nurse', 'selfie', 'pussy licking', 'yoga pants', 'pigtails', 'oiled', 'curly', 'leather', 'public', 'handjob', 'facial', 'pool', 'bath', 'sports', 'boots', 'ebony', 'kissing', 'fingering', 'jeans', 'bondage', 'bdsm', 'dress', 'bikini', 'socks', 'glasses', 'office', 'pantyhose', 'lesbian', 'shorts', 'latina', 'asian', 'cowgirl', 'doggy style', 'curvy', 'bbw', 'thick', 'chubby', 'uniform', 'cumshot', 'tattoo', 'armpit', 'redhead', 'sex', 'blowjob', 'dildo', 'sex toys', 'outdoor', 'rough sex', 'anal', 'mature', 'petite', 'hairy', 'masturbation', 'upskirt', 'stockings', 'feet', 'skinny', 'lingerie', 'nipples', 'skirt', 'shaved', 'undressing', 'milf', 'heels', 'blonde', 'tiny tits', 'close up', 'big tits', 'clothed', 'teen', 'panties', 'asshole', 'brunette', 'spreading', 'legs', 'pussy', 'solo', 'pawg', 'ass']

In [ ]:
# As an AI image tagging expert, analyze the given image containing explicit content. 
# Your task is to categorize the image based on the given list of categories between the <Categories> and </Categories> tags. 
# Ensure you understand the meaning of each category, including abbreviations, slang, and explicit terms.
# 
# <Categories>
# ['Hairy', ' Armpit', ' Latina', ' Tiny Tits', ' Undressing', ' Sexy', ' Amateur', ' Upskirt', ' Nipples', ' Socks', ' Pussy', ' Close Up', ' Legs', ' Lingerie', ' Dress', ' Panties', ' Ass', ' Babe', ' Brunette', ' Beautiful', ' Clothed', ' Spreading', ' Asshole', ' Solo', ' PAWG']
# </Categories>
# Step-by-Step Instructions:
# 	1.	Review the provided image: Carefully examine the image content.
# 	2.	Identify relevant categories: Refer to the list between  and  tags.
# 	3.	Assign categories: Select and assign as many relevant categories from the list as possible, ensuring they accurately reflect the image’s features or context.
# 	4.	Adhere to the list: Use only the categories specified in the list. Assign a maximum of 10 categories per image.
# 
# Important Note:
# If you include any category not specified in the list, you will incur a penalty of $10.
# 
# Output Template:
# The output should be structured as a python string with ',' between categories. Example:
# category1, category2, category3, ..., category10

In [ ]:
def get_balanced_df(og_df, max_samples=1000):
    def get_rows_with_label(dataframe, label):
        return dataframe[dataframe['labels'].apply(lambda x: label in x)]

    balanced_dfs = {label: pd.DataFrame() for label in label_counts.keys()}

    for label in tqdm(ascending_labels, total=len(ascending_labels), desc='Balancing dataset'):
        label_df = get_rows_with_label(og_df, label)

        label_df = resample(
            label_df,
            n_samples=max_samples,
            random_state=42
        )

        balanced_dfs[label] = label_df

    balanced_df = pd.concat(balanced_dfs).drop_duplicates(subset='file_path').reset_index(drop=True)

    return balanced_df

In [ ]:
small_balanced_df = get_balanced_df(df, max_samples=1000)
medium_balanced_df = get_balanced_df(df, max_samples=3000)
final_balanced_df = df.copy()

In [ ]:
small_balanced_df.to_csv('datasets/small_file_paths.csv')
medium_balanced_df.to_csv('datasets/medium_file_paths.csv')
final_balanced_df.to_csv('datasets/all_file_paths.csv')

# One hot encoding

In [ ]:
def dataframe_one_hot_encoding(dataframe: pd.DataFrame):
    all_labels = set(label for labels in dataframe['labels'] for label in labels)
    one_hot_encoded = pd.DataFrame()

    for label in tqdm(all_labels, total=len(all_labels), desc='One hot encoding'):
        one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)

    final_df = pd.concat([dataframe, one_hot_encoded], axis=1)
    final_df = final_df.drop(columns=['labels', ], axis=1)
    final_df.reset_index(drop=True, inplace=True)

    cols_sorted = sorted(list(final_df.columns))
    cols_sorted.remove("file_path")

    final_df = final_df[["file_path", *cols_sorted]]

    return final_df

In [ ]:
small_final_df = dataframe_one_hot_encoding(small_balanced_df)
medium_final_df = dataframe_one_hot_encoding(medium_balanced_df)
final_df = dataframe_one_hot_encoding(df)

In [ ]:
small_final_df.to_csv("datasets/small_one_hot.csv")
medium_final_df.to_csv("datasets/medium_one_hot.csv")
final_df.to_csv("datasets/all_one_hot.csv")

# AI GEN

In [ ]:
df.shape

In [ ]:
from galleries_mapping import *

In [ ]:
_mandatory_columns = [
    'age',
]
mandatory_columns = list(
    x.strip() for x in AI_GEN_TAGS[_mandatory_columns].values.flatten() if isinstance(x, str)
)
_negative_columns = [
    'negative', 'more_people'
]
negative_columns = list(
    x.strip() for x in AI_GEN_TAGS[_negative_columns].values.flatten() if isinstance(x, str)
)
tags_to_drop = {
    'blowjob', 'reality', 'hardcore', 'cumshot', 'cowgirl', 'ass fucking', 'doggy style', 'teen', 'lesbian', 'model',
    'big cock', 'anal', 'bdsm', 'bondage', 'fetish', 'face', 'kissing', 'handjob', 'facial', 'fingering', 'groupsex',
    'cum in mouth', 'old young', 'titjob', 'interracial', 'pussy licking', 'threesome', 'pov', 'femdom', 'christmas',
    'girlfriend', 'cosplay', 'facesitting', 'massage', 'deepthroat', 'strapon', 'cheating', 'humping', 'cum in pussy',
    'ass licking', 'creampie', 'ball licking', 'spanking', 'orgasm', 'double penetration', 'couple', 'family',
    'anal gape', 'bbc', 'party', 'schoolgirl', 'fisting', 'missionary', 'squirting', 'pissing', 'gangbang', 'old man',
    'ffm', 'cuckold', 'seduction', 'tribbing', 'orgy', 'flexible', 'cfnm', 'footjob', 'blowbang', 'pegging', 'pregnant',
    'swingers', 'gloryhole', 'caught', 'college', 'yoga', 'casting', 'stripper', 'step sister', 'voyeur', 'mmf',
    'bukkake', 'gyno', 'small cock', 'babysitter', 'cheerleader', 'cum swapping', 'bisexual', 'goth', 'braces', 'pawg',
    'pretty', 'pigtails', 'emo', 'latex', 'babe', 'step brother', 'twink', 'shemale', 'ballerina', 'twins', 'pornstar',
    'model', 'latex', 'emo', 'latex', 'babe', 'leather', 'pigtails', 'halloween', 'wedding',
    'tall', 'doctor', 'vintage'
}

In [ ]:
def filter_positives(row):
    labels = row.labels

    for L in labels:
        if L in mandatory_columns:
            return labels


def filter_negatives(row):
    labels = row.labels
    for L in labels:
        if L in negative_columns:
            # print(f"Label {L} in negative_columns.")
            return None
        if L in tags_to_drop:
            # print(f"Label {L} in tags_to_drop.")
            return None
        # print(f"Label {L} is allowed.")
    return labels


ai_df = df.copy()
ai_df['labels'] = ai_df.apply(filter_positives, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
print(ai_df.shape)

ai_df['labels'] = ai_df.apply(filter_negatives, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
print(ai_df.shape)

In [ ]:
ai_df.reset_index(inplace=True, drop=True)

In [ ]:
ai_df['labels_length'] = ai_df['labels'].apply(len)
ai_df = ai_df[ai_df['labels_length'] > 5]
ai_df.drop(['labels_length'], axis=1, inplace=True)
ai_df.reset_index(drop=True, inplace=True)
ai_df.to_csv('datasets/ai_gen_dataset_5_cats.csv')
print("5 cats df shape:", ai_df.shape)

ai_df['labels_length'] = ai_df['labels'].apply(len)
ai_df = ai_df[ai_df['labels_length'] > 10]
ai_df.drop(['labels_length'], axis=1, inplace=True)
ai_df.reset_index(drop=True, inplace=True)
ai_df.to_csv('datasets/ai_gen_dataset_10_cats.csv')
print("10 cats df shape:", ai_df.shape)

In [ ]:
all_labels = [label for labels in ai_df['labels'] for label in labels]
cntr = Counter(all_labels)
cntr

In [ ]:
ai_df.head()

In [ ]:
cats_5 = pd.read_csv('datasets/ai_gen_dataset_5_cats.csv', index_col=0)
cats_10 = pd.read_csv('datasets/ai_gen_dataset_10_cats.csv', index_col=0)
small_one_hot = pd.read_csv('datasets/small_one_hot.csv', index_col=0)
medium_one_hot = pd.read_csv('datasets/medium_one_hot.csv', index_col=0)

In [ ]:
for x in [cats_5, cats_10, small_one_hot, medium_one_hot]:
    missing = 0
    for file_path in x['file_path']:
        file_path = Path(file_path)
        json_path = SRC_DIR / file_path.parent / 'cropped' / (file_path.stem + '.json')
        if not json_path.exists():
            missing += 1

    print(f">>>>> Missing files: {missing} for out of {x.shape[0]}")